# Introduction to Brain Segmentation with Keras

# ***MAIN 2019 Educational Course***

## Thomas Funck

## McGill University

## **Contact**: email: [tffunck@gmail.com](mailto:tffunck@gmail.com) , Twitter: [@tffunck](https://twitter.com/tffunck)

In [3]:
#Download repository
!git clone https://github.com/tfunck/minc_keras
  
#Switch dir
import os
if os.getcwd() != '/content/minc_keras/' : os.chdir('/content/minc_keras')

!pwd

fatal: destination path 'minc_keras' already exists and is not an empty directory.
/content/minc_keras


In [5]:
#Unzip
!tar -jxvf data/output.tar.bz2 
!mv output mri

!wget https://amnesia.cbrain.mcgill.ca/deeplearning/sorteo.tar.bz2 --no-check-certificate
  
!mkdir -p pet
!tar -jxvf sorteo.tar.bz2 -C pet # &> /dev/null

output/sub-52783/
output/sub-64969/
output/sub-46541/
output/sub-26938/sub-26938_task-01_ses-01_T1w_anat_rsl.mnc
output/sub-61442/
output/sub-33393/sub-33393_task-01_ses-01_T1w_anat_rsl.mnc
output/sub-61442/sub-61442_task-01_ses-01_T1w_anat_rsl.mnc
output/sub-08001/sub-08001_task-01_ses-01_T1w_anat_rsl.mnc
output/sub-35634/sub-35634_task-01_ses-01_T1w_variant-seg_rsl.mnc
output/sub-50637/sub-50637_task-01_ses-01_T1w_variant-seg_rsl.mnc
output/sub-30072/sub-30072_task-01_ses-01_T1w_variant-seg_rsl.mnc
output/sub-59347/
output/sub-42555/sub-42555_task-01_ses-01_T1w_anat_rsl.mnc
output/sub-76160/sub-76160_task-01_ses-01_T1w_anat_rsl.mnc
output/sub-09644/sub-09644_task-01_ses-01_T1w_anat_rsl.mnc
output/sub-01903/sub-01903_task-01_ses-01_T1w_variant-seg_rsl.mnc
output/sub-29158/sub-29158_task-01_ses-01_T1w_variant-seg_rsl.mnc
output/sub-84858/sub-84858_task-01_ses-01_T1w_variant-seg_rsl.mnc
output/sub-18758/
output/sub-97823/
output/sub-39635/sub-39635_task-01_ses-01_T1w_variant-seg_rsl.mnc

# ***Outline***

# Simple Convolutional Neural Network

In [3]:
import minc_keras
from utils import *
import numpy as np
from minc_keras import *
### Set input and label string
input_str='_T1w.mnc' 
label_str='variant-seg'

### Set filename for .csv that will store data frame 
images_fn='mri.csv'

### Set source directory from which data will be read
source_dir="mri/"

### Set the target directory where output results will be saved
target_dir="mri_results"

### Set raiots for train/validation/test
ratios=[0.7,0.15]

### By default we set clobber to False so that we don't overwrite existing files
### Feel free to change if needed
clobber=False

### Size of batches that will be passed to model. The default 2 makes it easy
batch_size=2

### Image dimensions. We are slicing the 3D images into 2D slices. This serves to augment the data
### and make training faster
feature_dim=2


setup_dirs(target_dir)  

### Set filename for .csv file that will contain info about input images
images_fn = set_model_name(images_fn, report_dir, '.csv')

### Load data from brain images and save them into .npy. Sort them into train/validate/test splits
[images, data] = prepare_data(source_dir, data_dir, report_dir, input_str, label_str, ratios, batch_size,feature_dim, images_fn,  clobber=clobber)

### 1) Load data
Y_validate=np.load(data["validate_y_fn"]+'.npy')
nlabels=len(np.unique(Y_validate))
X_train=np.load(data["train_x_fn"]+'.npy')
Y_train=np.load(data["train_y_fn"]+'.npy')
X_validate=np.load(data["validate_x_fn"]+'.npy')

X_test=np.load(data["test_x_fn"]+'.npy')
Y_test=np.load(data["test_y_fn"]+'.npy')

Y_test=to_categorical(Y_test)
Y_train = to_categorical(Y_train, num_classes=nlabels)
Y_validate = to_categorical(Y_validate, num_classes=nlabels)

1 [92, 110, 92]
2 [92, 110, 92]


## Training a simple model

In [16]:
import keras
from keras.layers.convolutional import Conv2D
from keras.layers import Input
from custom_loss import *
from keras.utils import to_categorical
from keras.activations import relu
from keras.layers.core import Dropout
from keras.callbacks import History, ModelCheckpoint

model_name="mri_1.hdf5"

IN = Input(shape=(data["image_dim"][1], data["image_dim"][2],1))
CONV1 = Conv2D(16, kernel_size=[3,3], activation="relu",padding='same')(IN)
CONV2 = Conv2D(16, kernel_size=[3,3], activation="relu",padding='same')(CONV1)
CONV3 = Conv2D(16, kernel_size=[3,3], activation="relu",padding='same')(CONV2)
OUT = Conv2D(nlabels, kernel_size=[1,1], activation='softmax', padding='same')(CONV3)
model = keras.models.Model(inputs=[IN], outputs=OUT)

print(model.summary())

#set compiler
ada = keras.optimizers.Adam(0.0001)

#compile the model
model.compile(loss = 'categorical_crossentropy', optimizer=ada,metrics=['acc'] )
#fit model

history = model.fit([X_train],Y_train,  validation_data=([X_validate], Y_validate), epochs = 3)
#save model   
model.save(model_name)

test_score = model.evaluate(X_test, Y_test)
print("Test :", test_score)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 110, 92, 1)        0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 110, 92, 16)       160       
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 110, 92, 16)       2320      
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 110, 92, 16)       2320      
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 110, 92, 3)        435       
Total params: 5,235
Trainable params: 5,235
Non-trainable params: 0
_________________________________________________________________
None
Train on 23130 samples, validate on 5062 samples
Epoch 1/3
23130/23130 [==============================] - 34s 1ms/step - loss: 0.3799 -

In [0]:
###Create predictions for model
from predict import *
predict(model_name, test_dir, data_dir, report_dir+'/mri.csv', 'categorical_crossentropy', images_to_predict='all', category="test", verbose=True)

## Adding drop-out

In [0]:
import keras
from keras.layers.convolutional import Conv2D
from keras.layers import Input
from custom_loss import *
from keras.utils import to_categorical
from keras.activations import relu
from keras.layers.core import Dropout
from keras.callbacks import History, ModelCheckpoint

model_name="mri_1.hdf5"

IN = Input(shape=(data["image_dim"][1], data["image_dim"][2],1))
CONV1 = Conv2D(16, kernel_size=[3,3], activation="relu",padding='same')(IN)
DROPOUT1 = Dropout(0.25)(CONV1)
CONV2 = Conv2D(16, kernel_size=[3,3], activation="relu",padding='same')(DROPOUT1)
DROPOUT2 = Dropout(0.25)(CONV2)
CONV3 = Conv2D(16, kernel_size=[3,3], activation="relu",padding='same')(DROPOUT2)
DROPOUT3 = Dropout(0.25)(CONV3)
OUT = Conv2D(nlabels, kernel_size=[1,1], activation='softmax', padding='same')(DROPOUT3)
model = keras.models.Model(inputs=[IN], outputs=OUT)

print(model.summary())

#set compiler
ada = keras.optimizers.Adam(0.0001)

#compile the model
model.compile(loss = 'categorical_crossentropy', optimizer=ada,metrics=['acc'] )
#fit model

history = model.fit([X_train],Y_train,  validation_data=([X_validate], Y_validate), epochs = 3)
#save model   
model.save(model_name)

test_score = model.evaluate(X_test, Y_test)
print("Test :", test_score)


## Adding dilations

In [0]:
import keras
from keras.layers.convolutional import Conv2D
from keras.layers import Input
from custom_loss import *
from keras.utils import to_categorical
from keras.activations import relu
from keras.layers.core import Dropout
from keras.callbacks import History, ModelCheckpoint

model_name="mri_1.hdf5"

IN = Input(shape=(data["image_dim"][1], data["image_dim"][2],1))
CONV1 = Conv2D(16, kernel_size=[3,3], activation="relu",padding='same')(IN)
DROPOUT1 = Dropout(0.25)(CONV1)
CONV2 = Conv2D(16, kernel_size=[3,3], dilation_rate=[2,2], activation="relu",padding='same')(DROPOUT1)
DROPOUT2 = Dropout(0.25)(CONV2)
CONV3 = Conv2D(16, kernel_size=[3,3], dilation_rate=[3,3], activation="relu",padding='same')(DROPOUT2)
DROPOUT3 = Dropout(0.25)(CONV3)
OUT = Conv2D(nlabels, kernel_size=[1,1], activation='softmax', padding='same')(DROPOUT3)
model = keras.models.Model(inputs=[IN], outputs=OUT)

print(model.summary())

#set compiler
ada = keras.optimizers.Adam(0.0001)

#compile the model
model.compile(loss = 'categorical_crossentropy', optimizer=ada,metrics=['acc'] )
#fit model

history = model.fit([X_train],Y_train,  validation_data=([X_validate], Y_validate), epochs = 3)
#save model   
model.save(model_name)

test_score = model.evaluate(X_test, Y_test)
print("Test :", test_score)


## __Excercises__

### 1. Modify the drop-out rate and dilation_rate

### 2.  Build a CNN with: 5 convolutional layers, 5x5 kernels and 8 kernels in each layer

### 3. Train a CNN that has better than 0.96 test accuracy. How high can you get the accuracy without overfitting? 

## Solutions
### 2.


In [0]:
import keras
from keras.layers.convolutional import Conv2D
from keras.layers import Input
from custom_loss import *
from keras.utils import to_categorical
from keras.activations import relu
from keras.layers.core import Dropout


model_name="mri_1.hdf5"

IN = Input(shape=(data["image_dim"][1], data["image_dim"][2],1))
CONV1 = Conv2D(8, kernel_size=[5,5], activation="relu",padding='same')(IN)
CONV2 = Conv2D(8, kernel_size=[5,5], activation="relu",padding='same')(CONV1)
CONV3 = Conv2D(8, kernel_size=[5,5], activation="relu",padding='same')(CONV2)
CONV4 = Conv2D(8, kernel_size=[5,5], activation="relu",padding='same')(CONV3)
CONV5 = Conv2D(8, kernel_size=[5,5], activation="relu",padding='same')(CONV4)
OUT = Conv2D(nlabels, kernel_size=[1,1], activation='softmax', padding='same')(CONV5)
model = keras.models.Model(inputs=[IN], outputs=OUT)

print(model.summary())

#set compiler
ada = keras.optimizers.Adam(0.0001)

#compile the model
model.compile(loss = 'categorical_crossentropy', optimizer=ada,metrics=['acc'] )
#fit model

history = model.fit([X_train],Y_train,  validation_data=([X_validate], Y_validate), epochs = 3)
#save model   
model.save(model_name)

test_score = model.evaluate(X_test, Y_test)
print("Test :", test_score)


### 3) Run the previous model for 10 iterations to exceed 0.961 test accuracy.

#U-Net

![](https://github.com/tfunck/minc_keras/blob/master/images/splits_c_3.png?raw=1)

In [6]:
import minc_keras
from utils import *
import numpy as np
from minc_keras import *
### Set input and label string
input_str='_pet.mnc' 
label_str='brainmask'

### Set filename for .csv that will store data frame 
images_fn='pet.csv'

### Set source directory from which data will be read
source_dir="pet/"

### Set the target directory where output results will be saved
target_dir="pet_results"

### Set raiots for train/validation/test
ratios=[0.70,0.15]

### By default we set clobber to False so that we don't overwrite existing files
### Feel free to change if needed
clobber=True

### Size of batches that will be passed to model. The default 2 makes it easy
batch_size=2

### Image dimensions. We are slicing the 3D images into 2D slices. This serves to augment the data
### and make training faster
feature_dim=2

setup_dirs(target_dir)  

### Set filename for .csv file that will contain info about input images
images_fn = set_model_name(images_fn, report_dir, '.csv')

### Load data from brain images and save them into .npy. Sort them into train/validate/test splits
[images, data] = prepare_data(source_dir, data_dir, report_dir, input_str, label_str, ratios, batch_size,feature_dim, images_fn, pad_base=4,  clobber=clobber)

### 1) Load data
Y_validate=np.load(data["validate_y_fn"]+'.npy')
nlabels=len(np.unique(Y_validate))

X_train=np.load(data["train_x_fn"]+'.npy')
Y_train=np.load(data["train_y_fn"]+'.npy')
X_validate=np.load(data["validate_x_fn"]+'.npy')

X_test=np.load(data["test_x_fn"]+'.npy')
Y_test=np.load(data["test_y_fn"]+'.npy')

Y_test=to_categorical(Y_test)
Y_train = to_categorical(Y_train, num_classes=nlabels)
Y_validate = to_categorical(Y_validate, num_classes=nlabels)

train : expected/real ratio = 70.00 / 73.33
validate : expected/real ratio = 15.00 / 20.00


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


1 [100, 100, 100]
2 [100, 112, 112]





## Building a U-NET in Keras

![](https://github.com/tfunck/minc_keras/blob/master/images/unet.png?raw=1)

Ronneberger, Fischer, and Brox. 2015."U-net: Convolutional networks for biomedical image segmentation." International Conference on Medical image computing and computer-assisted intervention. https://arxiv.org/abs/1505.04597

In [8]:
import keras
from keras.layers.convolutional import Conv2D
from keras.layers import Input
from custom_loss import *
from keras.utils import to_categorical
from keras.activations import relu
from keras.layers.core import Dropout
from keras.layers import Input, Add, Multiply, Dense, BatchNormalization
from keras.layers import LeakyReLU, MaxPooling2D, Conv2DTranspose, Concatenate, ZeroPadding2D
from prepare_data import pad

### Warning : if you change the number of times you downsample with max_pool,
###           then you need to rerun prepare_data() with pad_base=<number of downsample nodes>
model_name="mri_1.hdf5"

#n_max_pool=4
#pad_y_dim = pad( data['image_dim'][1], n_max_pool) - data['image_dim'][1] +2 
#pad_x_dim = pad( data['image_dim'][2], n_max_pool) - data['image_dim'][2] +2
#print(pad_y_dim, pad_x_dim)
print(data['image_dim'])
### 1) Define architecture of neural network    
IN = Input(shape=(data['image_dim'][1],data['image_dim'][2] ,1))

BN1 = BatchNormalization()(IN)

conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(BN1)
conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
 
conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
 
conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool4)
conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)
    
up5 = UpSampling2D(size=(2, 2))(conv5)
#up5 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv5)
conc5 = Concatenate(axis=3)([up5, conv4])
conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conc5)
conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv6)

up6 = UpSampling2D(size=(2, 2))(conv6)
#up6 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv6)
conc6 = Concatenate(axis=3)([up6, conv3])
conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(up6)
conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv7)

up7 = UpSampling2D(size=(2, 2))(conv7)
#up7 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv7)
conc7 = Concatenate(axis=3)([up7, conv2])
conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conc7) #(up8)
conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

up8 = UpSampling2D(size=(2, 2))(conv8)
#up8 = Conv2DTranspose( filters=512, kernel_size=(3,3), strides=(2, 2), padding='same')(conv8)
conc8 = Concatenate(axis=3)([up8, conv1])
conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conc8)
conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

conv10 = Convolution2D(nlabels, 1, 1, activation='softmax')(conv9)

model = keras.models.Model(input=[IN], output=conv10)

print(model.summary())

#set compiler
ada = keras.optimizers.Adam(0.0001)

#compile the model
model.compile(loss = 'categorical_crossentropy', optimizer=ada,metrics=['acc'] )
#fit model

history = model.fit([X_train],Y_train,  validation_data=([X_validate], Y_validate), epochs = 3)
#save model   
model.save(model_name)

test_score = model.evaluate(X_test, Y_test)
print("Test :", test_score)



[100, 112, 112]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: Update your `Conv2D` 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 112, 112, 1)  0                                            
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 112, 112, 1)  4           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 112, 112, 32) 320         batch_normalization_2[0][0]      
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 112, 112, 32) 9248        conv2d_20[0][0]                  
__________________________________________________________________________________________________
max_poolin

In [0]:
###Create predictions for model
from predict import *
predict(model_name, test_dir, data_dir, report_dir+'/pet.csv', 'categorical_crossentropy', images_to_predict='all', category="test", verbose=True)


In [0]:
from google.colab import files
from glob import glob
for fn in glob('predict/*/*.png') :  files.download(fn)
import matplotlib.pyplot as plt
plt.imshow(plt.imread('pet_results/predict/test//sub-D13_ses-01_task-01_acq-rcl_dwn-smpl_pet_predict_2.png'))
